In [216]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import random as rnd

In [217]:
train_raw = pd.read_csv("data\\train.csv")
formula_train = pd.read_csv("data\\formula_train.csv")
test = pd.read_csv("data\\test.csv")
formula_test = pd.read_csv("data\\formula_test.csv")

train = train_raw.copy()

In [218]:
print(train.isna().sum().sum() + formula_train.isna().sum().sum() + test.isna().sum().sum() + formula_test.isna().sum().sum())

0


In [219]:
y = train['critical_temp']


formula_train.drop(columns=['critical_temp'], inplace=True)
formula_train.drop(columns=['material'], inplace=True)
train.drop(columns=['critical_temp'], inplace=True)
train_full = pd.concat([train, formula_train], axis=1)


x = train_full

formula_test.drop(columns=['material'], inplace=True)
test_full = pd.concat([test, formula_test], axis=1)


In [220]:

def egg(x, y, model=None):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    if model is None:
        model = LinearRegression()
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)

    print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', mse)

    return model

In [221]:
res1 = egg(x, y)

Mean Absolute Error: 12.604360514313852
Mean Squared Error: 285.83468667437154


In [222]:
def remove_outliers_iqr(df, columns, threshold=1.5):
    df_clean = df.copy()
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            df_clean = df_clean[(df_clean[col] >= lower_bound) & 
                                (df_clean[col] <= upper_bound)]
    
    return df_clean.reset_index(drop=True)


def drop_highcorr(df, thr=0.75):
    corr = df.corr().abs()
    to_drop = set()
    for i, col1 in enumerate(df.columns):
        for j, col2 in enumerate(df.columns[i+1:]):
            if corr[col1][col2] > thr:
                to_drop |= {col2}
    return df.drop(columns=list(to_drop))

In [223]:
train_full_1 = pd.concat([train_raw, formula_train], axis=1)
train_full_1 = remove_outliers_iqr(train_full_1, columns=train_raw.columns, threshold=1.5)

x1 = train_full_1.drop(columns=['critical_temp'])
y1 = train_full_1['critical_temp']

In [224]:
res5 = egg(x1, y1)

Mean Absolute Error: 14.690628157713352
Mean Squared Error: 383.0921541883483


In [225]:
res8 = egg(drop_highcorr(x1), y1)

Mean Absolute Error: 15.47637601453709
Mean Squared Error: 382.3955593357121


In [226]:
res6 = egg(x1.drop(columns=formula_train.columns), y1)

Mean Absolute Error: 14.80509481985076
Mean Squared Error: 357.8247960309648


In [227]:
res3 = egg(train, y)

Mean Absolute Error: 13.321268536555987
Mean Squared Error: 305.09808913552973


In [228]:
formula_train.drop(columns=[z for z in formula_train.columns if formula_train[z].sum() == 0.0], inplace=True)

In [229]:
new_train = drop_highcorr(train, 0.9)

In [230]:
res4 = egg(pd.concat([new_train, formula_train], axis=1), y)

Mean Absolute Error: 13.589087393183195
Mean Squared Error: 866.3740487188109


In [239]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('model', ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000))
])

res10 = egg(x, y, model=pipeline)



Mean Absolute Error: 13.26492409873573
Mean Squared Error: 305.9120131821184


Mean Absolute Error: 13.764831647725108
Mean Squared Error: 325.8802257660031


c:\Users\Иван\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.069e+06, tolerance: 1.416e+03
  model = cd_fast.enet_coordinate_descent(


In [240]:
# model = LinearRegression()
# model.fit(x, y)

y_pred_test = res10.predict(test_full)

y_pred_test = pd.DataFrame(y_pred_test, columns=['critical_temp'])
y_pred_test = y_pred_test.reset_index()

y_pred_test.to_csv("solutions\\solution.csv", index=False)

